# Predictive Model

#### Packages

In [46]:
# file manipualtion
from pathlib import Path
import numpy as np
import pandas as pd

# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

# balancing dataset
from imblearn.over_sampling import SMOTE

# models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

# visualistaion
import matplotlib.pyplot as plt

# warning supression
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

#### Directory  creation and file management

In [2]:
# defining the directory to original data
cwd = Path('./')
root_dir = cwd.resolve().parent

original_data_dir = root_dir / 'data'
additional_data_dir = root_dir / 'additional_data'

# list the .csv files for the project
for file in original_data_dir.glob('*.csv'):
    print(file)

/Volumes/GoogleDrive/My Drive/Dev/TrafficAccidents/data/accidents2019.csv
/Volumes/GoogleDrive/My Drive/Dev/TrafficAccidents/data/casualties2019.csv
/Volumes/GoogleDrive/My Drive/Dev/TrafficAccidents/data/vehicles2019.csv


#### Testing classification models

In [3]:
def get_models():
    models = {}
    models['decision_tree'] = DecisionTreeClassifier()
    models['knn'] = KNeighborsClassifier()
    models['logistic_regression'] = LogisticRegression()
    models['bayes'] = MultinomialNB()
    models['rf'] = RandomForestClassifier()
    return models

models = get_models()

models

{'decision_tree': DecisionTreeClassifier(),
 'knn': KNeighborsClassifier(),
 'logistic_regression': LogisticRegression(),
 'bayes': MultinomialNB(),
 'rf': RandomForestClassifier()}

In [4]:
# import cleaned accidents data frame from the pickle file
accidents = pd.read_pickle(additional_data_dir / 'accidents_cleaned.pkl')
vehicles = pd.read_csv(original_data_dir / 'vehicles2019.csv')
casualties = pd.read_csv(original_data_dir / 'casualties2019.csv')

# convert column names to lowercase for ease of indexing
def lower_columns(df):
    """
    Defintion:
        convert column names to lower case
    """
    df.columns = map(str.lower, df.columns)
    
# converting all column names to lower case
lower_columns(vehicles)
lower_columns(casualties)

accidents = pd.merge(accidents, vehicles, on='accident_index')
accidents = pd.merge(accidents, casualties, on='accident_index')
accidents.columns

/var/folders/ym/67nh3yg92pl8g2wbm_vrs8_h0000gn/T/ipykernel_21221/1089652144.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles = pd.read_csv(original_data_dir / 'vehicles2019.csv')
/var/folders/ym/67nh3yg92pl8g2wbm_vrs8_h0000gn/T/ipykernel_21221/1089652144.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  casualties = pd.read_csv(original_data_dir / 'casualties2019.csv')


Index(['accident_index', 'longitude', 'latitude', 'police_force',
       'accident_severity', 'number_of_vehicles', 'number_of_casualties',
       'day_of_week', 'local_authority_(district)',
       'local_authority_(highway)', '1st_road_class', '1st_road_number',
       'road_type', 'speed_limit', 'junction_detail', 'junction_control',
       '2nd_road_class', '2nd_road_number',
       'pedestrian_crossing-human_control',
       'pedestrian_crossing-physical_facilities', 'light_conditions',
       'weather_conditions', 'road_surface_conditions',
       'special_conditions_at_site', 'carriageway_hazards',
       'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident',
       'lsoa_of_accident_location', 'district', 'converted_date',
       'converted_time', 'datetime', 'decimal_time', 'day_of_year',
       'vehicle_reference_x', 'vehicle_type', 'towing_and_articulation',
       'vehicle_manoeuvre', 'vehicle_location-restricted_lane',
       'junction_location', 'skidding_a

# Feature Selection

### Test for columns with many values less than 0

Any column with more than 10% of values being less than 0 is dropped.

In [5]:
# exclude columns with non-numerical data types
test = accidents.select_dtypes(exclude=['object', 'datetime64'])

ratios = ((test < 0).sum() / test.sum()).to_dict()

to_delete = []
for k, v in ratios.items():
    if v > 0.10:
        to_delete.append(k)

print(to_delete)

['junction_control', '2nd_road_class', 'towing_and_articulation', 'propulsion_code', 'driver_home_area_type', 'casualty_home_area_type']


In [6]:
# manually determine data type of features
numeric = ['number_of_vehicles', 'number_of_casualties', 'age_of_driver',
           'engine_capacity_(cc)', 'age_of_vehicle', 'age_of_casualty']

binary = ['was_vehicle_left_hand_drive?', 'sex_of_driver', 'sex_of_casualty']

nominal = ['1st_road_class', 'road_type', 'junction_detail', 'light_conditions',
           'weather_conditions', 'road_surface_conditions',
           'urban_or_rural_area',
           'vehicle_type', 'vehicle_manoeuvre',
           'junction_location', 'journey_purpose_of_driver', 'casualty_type']

ordinal = ['speed_limit', 'day_of_week', 'day_of_year', 'age_band_of_driver', 'age_band_of_casualty']

target = 'accident_severity'

In [57]:
class FeatureSelection:

    def __init__(self, df, cat_features, numeric_features, target):
        self.df = df
        self.cat_features = cat_features
        self.numeric_features = numeric_features
        self.target = target

    def remove_negatives(self):
        features = self.numeric_features + self.cat_features
        filtered = self.df.filter(features + [self.target], axis=1)
        filtered = filtered[(filtered > 0).all(1)]
        return filtered

    def get_feature_scores(self, dtype='cat', k='all'):
        filtered_dataset = self.remove_negatives()
        if dtype == 'cat':
            features = self.cat_features
            selector = SelectKBest(f_classif, k=k)
        if dtype == 'num':
            features = self.numeric_features
            selector = SelectKBest(f_classif, k=k)
        selector.fit(filtered_dataset[features], filtered_dataset[self.target])

        cols = selector.get_support(indices=True)
        p_values = selector.pvalues_

        p_values = pd.Series(p_values, dtype='object')
        scores = -np.log10(selector.pvalues_)
        return cols, scores

    def plot_features(self):
        cat_features, cat_scores = self.get_feature_scores(dtype='cat', k='all')
        num_features, num_scores = self.get_feature_scores(dtype='num', k='all')
        features = cat_features + num_features
        scores = list(cat_scores) + list(num_scores)
        plt.bar(range(len(features)), scores)
        plt.xticks(range(len(features)), features, rotation='vertical')
        plt.show()
    
    def get_new_X(self):
        filtered_dataset = self.remove_negatives()
        cat, _ = self.get_feature_scores(dtype='cat', k=12)
        numeric, _ = self.get_feature_scores(dtype='num', k=2)
        target = self.target
        cols = np.concatenate((cat, numeric))
        target_i = filtered_dataset.columns.get_loc(self.target)
        cols = np.append(cols, target_i)
        new_df = filtered_dataset.iloc[:, cols]
        
        return new_df

fs = FeatureSelection(accidents, nominal+ordinal+binary, numeric, target)

filtered = fs.get_new_X()


# Constructing the model

We see that the model is heavily imbalanced towards accident_severity label 3. 

Hence, SMOTE oversampling should be used to balance the dataset for more accurate training.

In [58]:
filtered.accident_severity.value_counts()

3    31739
2     7121
1      602
Name: accident_severity, dtype: int64

In [64]:
X_data = filtered.iloc[:, :-1]
y_data = filtered.iloc[:, -1]

print("Shapes prior to over-sampling: \n")
print(f"X_data shape: {X_data.shape}")
print(f"y_data shape: {y_data.shape}")

oversample = SMOTE()
X_data, y_data = oversample.fit_resample(X_data, y_data)

print("\n\nShapes after over-sampling: \n")
print(f"X_data shape: {X_data.shape}")
print(f"y_data shape: {y_data.shape}")

Shapes prior to over-sampling: 

X_data shape: (39462, 14)
y_data shape: (39462,)


Shapes after over-sampling: 

X_data shape: (95217, 14)
y_data shape: (95217,)


In [65]:
print("Label ratio after over-sampling")
y_data.value_counts()

Label ratio after over-sampling


3    31739
1    31739
2    31739
Name: accident_severity, dtype: int64

In [61]:
# splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25)

# data set shapes
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (71412, 14)
y_train shape: (71412,)
X_test shape: (23805, 14)
y_test shape: (23805,)


#### Evaluate model

Here we use a repeated stratified k-fold cross-validation method, so that the training data is split into k folds with equal proportions per label. Each k-1 folds are evaluated using the final fold as validation, and the process is repeated three times, hence repeated stratified k-fold.

Defining a stacked model and appending to the list of individual models

In [66]:
def get_stacking():
    level0 = []
    for k, v in models.items():
        level0.append((k, v))
    level1 = LogisticRegression()
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

models['stack'] = get_stacking()

get_stacking()

StackingClassifier(cv=5,
                   estimators=[('decision_tree', DecisionTreeClassifier()),
                               ('knn', KNeighborsClassifier()),
                               ('logistic_regression', LogisticRegression()),
                               ('bayes', MultinomialNB()),
                               ('rf', RandomForestClassifier())],
                   final_estimator=LogisticRegression())

### Training all models in the models dictionary

In [70]:
models

{'decision_tree': DecisionTreeClassifier(),
 'knn': KNeighborsClassifier(),
 'logistic_regression': LogisticRegression(),
 'bayes': MultinomialNB(),
 'rf': RandomForestClassifier(),
 'stack': StackingClassifier(cv=5,
                    estimators=[('decision_tree', DecisionTreeClassifier()),
                                ('knn', KNeighborsClassifier()),
                                ('logistic_regression', LogisticRegression()),
                                ('bayes', MultinomialNB()),
                                ('rf', RandomForestClassifier())],
                    final_estimator=LogisticRegression())}

In [67]:
results, names, summary_results = [], [], []
for name, model in models.items():
    scores = evaluate_model(model, X_train, y_train)
    results.append(scores)
    names.append(name)
    summary_results.append({name: {'mean': np.mean(results), 'std': np.std(results)}})
    print(f"{name} {np.mean(scores):.4f}, {np.std(scores):.4f}")

decision_tree 0.8960, 0.0035
knn 0.8720, 0.0038
logistic_regression 0.4843, 0.0073
bayes 0.4019, 0.0075
rf 0.9352, 0.0035
stack 0.9483, 0.0028


In [68]:
summary_results

[{'decision_tree': {'mean': 0.8960398812524506, 'std': 0.003458774095393027}},
 {'knn': {'mean': 0.884020426072182, 'std': 0.012555773525351805}},
 {'logistic_regression': {'mean': 0.7507701786814541,
   'std': 0.18876942419494633}},
 {'bayes': {'mean': 0.6635579454433428, 'std': 0.2226147857167037}},
 {'rf': {'mean': 0.7178952183573255, 'std': 0.2268446018975331}},
 {'stack': {'mean': 0.7562975721478494, 'std': 0.2241809224705417}}]

An individual decision tree model has the highest accuracy of all the models tested at 89.6%.